In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
#import training dataset
dataset = pd.read_csv('tcd-ml-1920-group-income-train.csv',low_memory=False)
dataset = dataset.drop('Instance',axis=1)
#dataset = dataset.drop('Wears Glasses',axis=1)
#dataset = dataset.drop('Hair Color',axis=1)
#dataset = dataset.drop('Gender',axis=1)
#data_raw = dataset.drop(['Profession','Size of City'],axis=1)
dataset.info()

In [ ]:
#Cleaning training dataset
dataset['Housing Situation']=dataset['Housing Situation'].replace('0','unknown')
dataset['Housing Situation']=dataset['Housing Situation'].replace('nA','unknown')
dataset['Satisfation with employer']=dataset['Satisfation with employer'].fillna('unknown')
#dataset['Gender'] = dataset['Gender'].fillna('unknown')
#dataset['Gender']=dataset['Gender'].replace('0','unknown')
dataset['Gender'] = dataset['Gender'].fillna('male')
dataset['Gender']=dataset['Gender'].replace('0','male')
dataset['Profession']=dataset['Profession'].fillna('unknown')
dataset['University Degree']=dataset['University Degree'].fillna('unknown')
dataset['University Degree']=dataset['University Degree'].replace('0','unknown')
dataset['Hair Color']=dataset['Hair Color'].fillna('unknown')
dataset['Year of Record']=dataset['Year of Record'].fillna(dataset['Year of Record'].median())
dataset=dataset.drop(columns=['Hair Color','Profession','Wears Glasses','Crime Level in the City of Employement'])
dataset['Work Experience in Current Job [years]']=dataset['Work Experience in Current Job [years]'].replace('#NUM!',-1)
dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'])
dataset['Work Experience in Current Job [years]']=dataset['Work Experience in Current Job [years]'].replace(-1,dataset['Work Experience in Current Job [years]'].median())

In [ ]:
#Importing test data
test3 = pd.read_csv("tcd-ml-1920-group-income-test.csv",low_memory=False)

In [ ]:
#cleaning test dataset
test3['Housing Situation']=test3['Housing Situation'].replace('0','unknown')
test3['Housing Situation']=test3['Housing Situation'].replace('nA','unknown')
test3['Satisfation with employer']=test3['Satisfation with employer'].fillna('unknown')
#test3['Gender'] = test3['Gender'].fillna('unknown')
#test3['Gender']=test3['Gender'].replace('0','unknown')
test3['Gender'] = test3['Gender'].fillna('male')
test3['Gender']=test3['Gender'].replace('0','male')
test3['Profession']=test3['Profession'].fillna('unknown')
test3['University Degree']=test3['University Degree'].fillna('unknown')
test3['University Degree']=test3['University Degree'].replace('0','unknown')
test3['Hair Color']=test3['Hair Color'].fillna('unknown')
test3['Year of Record']=test3['Year of Record'].fillna(dataset['Year of Record'].median())
test3=test3.drop(columns=['Hair Color','Profession','Instance','Wears Glasses','Crime Level in the City of Employement'])
test3['Work Experience in Current Job [years]']=test3['Work Experience in Current Job [years]'].replace('#NUM!',-1)
test3['Work Experience in Current Job [years]']=pd.to_numeric(test3['Work Experience in Current Job [years]'])
test3['Work Experience in Current Job [years]']=test3['Work Experience in Current Job [years]'].replace(-1,dataset['Work Experience in Current Job [years]'].median())
test3['Country'] = test3['Country'].fillna('unknown')

In [ ]:
dataset1=dataset

In [ ]:
#Target Encoding
'''
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)
'''

In [ ]:
#COmbining test and train dataset for encoding
dataset['flag'] = 1
test3['flag'] = 0
data = pd.concat([dataset, test3], ignore_index=True)
data=data.drop('Total Yearly Income [EUR]',axis=1)
data

In [ ]:
#Encoding and seperating test and train data
data = pd.get_dummies(data)
dataset = data[data['flag'] == 1]
test3 = data[data['flag'] == 0]

In [ ]:
#Creating Predictor and response variable
X = dataset.drop(['flag'], axis=1)
y = dataset1.iloc[:,-2]
y=y.apply(np.log)

In [ ]:
#Validation and Train dataset
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

In [ ]:
#categorical_features_indices = [1,3,4,6,8,9]

In [ ]:
#Training Model
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=2000, depth=7, learning_rate=0.2, loss_function='MAE')
model.fit(X_train, y_train,eval_set=(X_validation, y_validation),plot=True)

In [ ]:
#Preparing Submission file
test1 = pd.read_csv("tcd-ml-1920-group-income-test.csv",low_memory=False)
submission = pd.DataFrame()
submission['Instance'] = test1['Instance']
submission['Total Yearly Income [EUR]'] = np.exp(model.predict(test3))
submission.to_csv("Catboost_model.csv")